In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import parallel_backend
from sklearn.metrics import accuracy_score
cars = pd.read_csv('car_prices.csv')

In [2]:
#Displaying Data 
cars.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [3]:
cars.tail()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
558832,2015,Kia,K900,Luxury,Sedan,NaN,knalw4d4xf6019304,in,45.0,18255.0,silver,black,avis corporation,35300.0,33000.0,Thu Jul 09 2015 07:00:00 GMT-0700 (PDT)
558833,2012,Ram,2500,Power Wagon,Crew Cab,automatic,3c6td5et6cg112407,wa,5.0,54393.0,white,black,i -5 uhlmann rv,30200.0,30800.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558834,2012,BMW,X5,xDrive35d,SUV,automatic,5uxzw0c58cl668465,ca,48.0,50561.0,black,black,financial services remarketing (lease),29800.0,34000.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558835,2015,Nissan,Altima,2.5 S,sedan,automatic,1n4al3ap0fc216050,ga,38.0,16658.0,white,black,enterprise vehicle exchange / tra / rental / t...,15100.0,11100.0,Thu Jul 09 2015 06:45:00 GMT-0700 (PDT)
558836,2014,Ford,F-150,XLT,SuperCrew,automatic,1ftfw1et2eke87277,ca,34.0,15008.0,gray,gray,ford motor credit company llc pd,29600.0,26700.0,Thu May 28 2015 05:30:00 GMT-0700 (PDT)


In [4]:
cars.describe()

,year,condition,odometer,mmr,sellingprice
count,558837.000000,547017.000000,558743.000000,558799.000000,558825.000000
mean,2010.038927,30.672365,68320.017767,13769.377495,13611.358810
std,3.966864,13.402832,53398.542821,9679.967174,9749.501628
min,1982.000000,1.000000,1.000000,25.000000,1.000000
25%,2007.000000,23.000000,28371.000000,7100.000000,6900.000000
50%,2012.000000,35.000000,52254.000000,12250.000000,12100.000000
75%,2013.000000,42.000000,99109.000000,18300.000000,18200.000000
max,2015.000000,49.000000,999999.000000,182000.000000,230000.000000


In [5]:
#Cleaning data
cleanedcars = cars.dropna()
cleanedcars

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558831,2011,BMW,5 Series,528i,Sedan,automatic,wbafr1c53bc744672,fl,39.0,66403.0,white,brown,lauderdale imports ltd bmw pembrok pines,20300.0,22800.0,Tue Jul 07 2015 06:15:00 GMT-0700 (PDT)
558833,2012,Ram,2500,Power Wagon,Crew Cab,automatic,3c6td5et6cg112407,wa,5.0,54393.0,white,black,i -5 uhlmann rv,30200.0,30800.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558834,2012,BMW,X5,xDrive35d,SUV,automatic,5uxzw0c58cl668465,ca,48.0,50561.0,black,black,financial services remarketing (lease),29800.0,34000.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558835,2015,Nissan,Altima,2.5 S,sedan,automatic,1n4al3ap0fc216050,ga,38.0,16658.0,white,black,enterprise vehicle exchange / tra / rental / t...,15100.0,11100.0,Thu Jul 09 2015 06:45:00 GMT-0700 (PDT)


In [6]:
cleanedcars.drop(['model', 'trim'], axis=1)

,year,make,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558831,2011,BMW,Sedan,automatic,wbafr1c53bc744672,fl,39.0,66403.0,white,brown,lauderdale imports ltd bmw pembrok pines,20300.0,22800.0,Tue Jul 07 2015 06:15:00 GMT-0700 (PDT)
558833,2012,Ram,Crew Cab,automatic,3c6td5et6cg112407,wa,5.0,54393.0,white,black,i -5 uhlmann rv,30200.0,30800.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558834,2012,BMW,SUV,automatic,5uxzw0c58cl668465,ca,48.0,50561.0,black,black,financial services remarketing (lease),29800.0,34000.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558835,2015,Nissan,sedan,automatic,1n4al3ap0fc216050,ga,38.0,16658.0,white,black,enterprise vehicle exchange / tra / rental / t...,15100.0,11100.0,Thu Jul 09 2015 06:45:00 GMT-0700 (PDT)


In [7]:
#checking to see if the dataframe has been cleaned 
cleanedcars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 472325 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          472325 non-null  int64  
 1   make          472325 non-null  object 
 2   model         472325 non-null  object 
 3   trim          472325 non-null  object 
 4   body          472325 non-null  object 
 5   transmission  472325 non-null  object 
 6   vin           472325 non-null  object 
 7   state         472325 non-null  object 
 8   condition     472325 non-null  float64
 9   odometer      472325 non-null  float64
 10  color         472325 non-null  object 
 11  interior      472325 non-null  object 
 12  seller        472325 non-null  object 
 13  mmr           472325 non-null  float64
 14  sellingprice  472325 non-null  float64
 15  saledate      472325 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 61.3+ MB


In [8]:
cleanedcars.isna().sum()

year            0
make            0
model           0
trim            0
body            0
transmission    0
vin             0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
mmr             0
sellingprice    0
saledate        0
dtype: int64

In [1]:
#One Hot Encoding 
#One Hot Encoding 
#No. of unique values -> {body: 84} {color: 19} {condition: 41} {transmission: 2} {year: 26} {make: 53}
#No. of unique values -> {trim: 1490} {odometer: 157707} {interior: 16} {seller: 11711}

bodydummy=pd.get_dummies(pdSalesData['body'],dtype=int)
print(bodydummy)

colordummy=pd.get_dummies(pdSalesData['color'],dtype=int)
print(colordummy)

transmissiondummy=pd.get_dummies(pdSalesData['transmission'],dtype=int)
print(transmissiondummy)

makedummy=pd.get_dummies(pdSalesData['make'],dtype=int)
print(makedummy)

interiordummy=pd.get_dummies(pdSalesData['interior'],dtype=int)
print(interiordummy)



NameError: name 'pd' is not defined

In [50]:
#Beginning the regression analysis 

y = cleanedcars['mmr']

features = ['year', 'odometer', 'condition']
X = cleanedcars[features]

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
print(X_train) 

        year  odometer  condition
51686   2013   46887.0       49.0
10090   2011   33309.0       25.0
555324  2014   18701.0       42.0
189884  2012   33574.0       36.0
554447  2009   75696.0       35.0
...      ...       ...        ...
310022  2012   34183.0       47.0
433048  2006  133159.0       35.0
164717  2013   39710.0       41.0
181922  2006  163172.0       25.0
153201  2011   34011.0        4.0

[377860 rows x 3 columns]


In [53]:
print(X_test) 

        year  odometer  condition
297590  2006  121222.0       34.0
90053   2010   42814.0       46.0
341051  2014   20032.0       42.0
362239  2013   52864.0       23.0
480321  2012   72306.0       37.0
...      ...       ...        ...
228828  2008  119936.0       19.0
494560  2013   30347.0       47.0
552996  2012   34831.0       38.0
272261  2014   30155.0       41.0
149526  2009   59027.0       19.0

[94465 rows x 3 columns]


In [54]:
print(y_train) 

51686     32200.0
10090     24300.0
555324    20200.0
189884    15050.0
554447     5250.0
           ...   
310022    12100.0
433048     8400.0
164717    20300.0
181922     2475.0
153201    22900.0
Name: mmr, Length: 377860, dtype: float64


In [55]:
print(y_test) 

297590     4150.0
90053     20400.0
341051    22900.0
362239    12050.0
480321     8725.0
           ...   
228828    19650.0
494560    20100.0
552996    25900.0
272261    13050.0
149526     9900.0
Name: mmr, Length: 94465, dtype: float64


In [58]:
#Linear Regression with the numerical features 
model =  LinearRegression().fit(X,y)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(y_pred)
print(f'Linear Regression MSE: {mean_squared_error(y_test, y_pred)}')
print(f'Linear Regression R² score: {r2_score(y_test, y_pred)}')

[ 7495.4575188  15805.78154307 20149.39903506 ... 17477.19270044
 19525.54324293 12689.36317869]
Linear Regression MSE: 54219061.07002882
Linear Regression R² score: 0.39420153919361567


In [61]:
#Beginning the regression analysis with more relevant features 

y1 = cleanedcars['mmr']

features = ['year', 'make', 'body','state','condition', 'odometer']

X1 = cleanedcars[features]

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [62]:
#Linear Regression with more relavant features 

# Define categorical features
cat_features = ['make', 'body', 'state']

preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)])

model1 = Pipeline(steps=[('preprocessor', preprocessor),('regressor', LinearRegression())])

# Fit the model to the training data
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)


linear_mse = mean_squared_error(y_test, y_pred1)
linear_r2 = r2_score(y_test, y_pred1)

# Print the predicted values and evaluation metrics
print("Predicted values:")
print(y_pred1)
print(f'Linear Regression MSE: {linear_mse}')
print(f'Linear Regression R² score: {linear_r2}')

Predicted values:
[10294.17913267 17482.4577905  16106.4201171  ... 24795.74626775
 12362.0756843  10762.39475615]
Linear Regression MSE: 60832884.83399991
Linear Regression R² score: 0.32030420166717954


In [18]:
#Beginning the regression analysis with all the features

y2 = cleanedcars['sellingprice']

# Define all features
features = ['year', 'make', 'body', 'transmission', 'state', 'condition', 'odometer', 'color', 'interior', 'seller', 'mmr']

# Select features for X2
X2 = cleanedcars[features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [1]:
#Linear Regression with all the features 

#Linear Regression with more relavant features

#I omitted vin because it is unique for each value, which means that it can't be used to predict anything, same with sale date 

# Define categorical features
cat_features2 = ['make', 'body','transmission','state','color','interior','seller','saledate']

preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_features2)])

model2 = Pipeline(steps=[('preprocessor', preprocessor),
                         ('regressor', LinearRegression())])

# Fit the model to the training data
model2.fit(X_train, y_train)
y_pred2 = model1.predict(X_test)


linear_mse1 = mean_squared_error(y_test, y_pred2)
linear_r22 = r2_score(y_test, y_pred2)

# Print the predicted values and evaluation metrics
print("Predicted values:")
print(y_pred1)
print(f'Linear Regression MSE: {linear_mse1}')
print(f'Linear Regression R² score: {linear_r22}')

NameError: name 'ColumnTransformer' is not defined

In [ ]:
#Support Vector Machine with the numerical features 
model1 = SGDRegressor().fit(X,y)

In [ ]:
with parallel_backend('threading', n_jobs=8):
    model1.fit(X_train, y_train)

In [ ]:
y_pred1 = model1.predict(X_test)

svm_mse = mean_squared_error(y_test, y_pred1)
svm_r2 = r2_score(y_test, y_pred1)

# Print the predicted values and evaluation metrics
print("Predicted values:")
print(y_pred1)
print(f'Stochastic Gradient Descent MSE: {svm_mse}')
print(f'Stoachastic Gradient Descent R² score: {svm_r2}')

In [ ]:
#Support Vector Machine with more relavant features 

In [ ]:
#Support Vector Machine with all the features 

In [ ]:
#Stochastic Gradient Descent with numerical features 
model2 =  SVR().fit(X,y)

In [ ]:
model.fit2(X_train, y_train)

In [ ]:
y_pred2 = model2.predict(X_test)
print(y_pred2)
print(f'Support Vector Machine MSE: {svm_mse}')
print(f'Support Vector Machine R² score: {svm_r2}')

In [ ]:
#Stoachstic Gradient Descent with more relavant features 

In [ ]:
#Stoachstic Gradient Descent with all the featuers 